In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import json
from contextlib import contextmanager
from pathlib import Path
import functools

from tqdm import tqdm
from einops import rearrange, reduce, einsum
import plotly.express as px
import plotly.graph_objects as go
import torch
from cupbearer import detectors, tasks, utils, scripts
from torch import Tensor, nn

/home/jordan/miniconda3/envs/MAD/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
task = tasks.tiny_natural_mechanisms("hex", device="cuda")

Loaded pretrained model attn-only-1l into HookedTransformer
Moving model to device:  cuda


/mnt/c/Users/nadro/Documents/AI_safety/CHAI/MAD_code/cupbearer/src/cupbearer/tasks/tiny_natural_mechanisms.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict =

In [10]:
with open(".cupbearer_cache/tiny_natural_mechanisms/hex_task.json") as f:
    task_config = json.load(f)

# cpu to avoid MPS issues with tensors that are too large
effect_tokens = torch.tensor(
    task_config["effect_tokens"], dtype=torch.long, device="cpu"
)

In [11]:
for name, _ in task.model.named_modules():
    print(name)


embed
hook_embed
pos_embed
hook_pos_embed
blocks
blocks.0
blocks.0.ln1
blocks.0.ln1.hook_scale
blocks.0.ln1.hook_normalized
blocks.0.attn
blocks.0.attn.hook_k
blocks.0.attn.hook_q
blocks.0.attn.hook_v
blocks.0.attn.hook_z
blocks.0.attn.hook_attn_scores
blocks.0.attn.hook_pattern
blocks.0.attn.hook_result
blocks.0.hook_attn_in
blocks.0.hook_q_input
blocks.0.hook_k_input
blocks.0.hook_v_input
blocks.0.hook_mlp_in
blocks.0.hook_attn_out
blocks.0.hook_mlp_out
blocks.0.hook_resid_pre
blocks.0.hook_resid_post
ln_final
ln_final.hook_scale
ln_final.hook_normalized
unembed


# Defining the detector

In [12]:
def effect_prob_func(logits):
    assert logits.ndim == 3
    probs = logits.softmax(-1)
    # Sum over vocab (but not batch) dim:
    return probs[:, -1, effect_tokens].sum(-1)

In [20]:
class GradientWeightedDetector(detectors.AnomalyDetector):
    def __init__(self, names: list[str], output_func_for_grads):
        super().__init__()
        self.names = names
        self.output_func_for_grads = output_func_for_grads

    def _train(
        self,
        trusted_data: torch.utils.data.Dataset,
        untrusted_data: torch.utils.data.Dataset | None,
        save_path: Path | str | None,
        batch_size: int = 64,
        **kwargs,
    ):
        assert trusted_data is not None

        data_loader = torch.utils.data.DataLoader(
            trusted_data, batch_size=batch_size, shuffle=False
        )
        example_batch = next(iter(data_loader))
        example_inputs = utils.inputs_from_batch(example_batch)
        example_activations = utils.get_activations(
            self.model, self.names, example_inputs
        )

        # v is an entire batch, v[0] are activations for a single input
        activation_sizes = {k: v[0].size() for k, v in example_activations.items()}
        device = next(iter(example_activations.values())).device

        for k, size in activation_sizes.items():
            if len(size) not in (1, 2):
                raise ValueError(
                    f"Activation size {size} of {k} is not supported. "
                    "Activations must be either 1D or 2D (in which case separate "
                    "covariance matrices are learned along the first dimension)."
                )
        self.means = {
            k: torch.zeros(size[-1], device=device)
            for k, size in activation_sizes.items()
        }
        self.Pi = {
            k: torch.zeros((size[-1], size[-1]), device=device)
            for k, size in activation_sizes.items()
        }
        self._n = 0

        for i, batch in enumerate(tqdm(data_loader)):
            inputs = utils.inputs_from_batch(batch).to(device)
            activations, grads = utils.get_activations_and_grads(
                self.model, self.names, self.output_func_for_grads, inputs
            )

            for k in self.names:
                # Flatten the activations to (batch, dim)
                activation = activations[k]
                grad = grads[k]

                assert activation.shape == grad.shape

                if activation.ndim == 3:
                    activation = rearrange(
                        activation, "batch independent dim -> (batch independent) dim"
                    )
                    grad = rearrange(
                        grad, "batch independent dim -> (batch independent) dim"
                    )
                assert activation.ndim == 2, activation.shape

                new_n = len(activation)
                total_n = self._n + new_n

                new_mean = activation.mean(dim=0)
                self.means[k] = (self._n * self.means[k] + new_n * new_mean) / total_n

                # Compute outer product, then take the mean over the batch dimension
                new_C = torch.einsum("bi, bj -> ij", grad, grad) / new_n
                self.Pi[k] = (self.Pi[k] * self._n + new_C * new_n) / total_n

                self._n = total_n

    def _compute_layerwise_scores(self, batch):
        inputs = utils.inputs_from_batch(batch).to("mps")

        activations, grads = utils.get_activations_and_grads(
            self.model, self.names, self.output_func_for_grads, inputs
        )
        batch_size = next(iter(activations.values())).shape[0]

        distances: dict[str, torch.Tensor] = {}
        # Reshape activations to (batch, dim) for computing distances
        for k in self.names:
            activation = activations[k]
            grad = grads[k]
            assert activation.shape == grad.shape
            if activation.ndim == 3:
                activation = rearrange(
                    activation, "batch independent dim -> (batch independent) dim"
                )
                grad = rearrange(
                    grad, "batch independent dim -> (batch independent) dim"
                )
            assert activation.ndim == 2, activation.shape

            delta = activation - self.means[k]
            distance = torch.einsum("bi,ij,bj->b", delta, self.Pi[k], delta)
            distances[k] = distance

        for k, v in distances.items():
            # Unflatten distances so we can take the mean over the independent axis
            distances[k] = rearrange(
                v, "(batch independent) -> batch independent", batch=batch_size
            ).mean(dim=1)

        return distances

    def _get_trained_variables(self, saving: bool = False):
        return {
            "means": self.means,
            "Pi": self.Pi,
        }

    def _set_trained_variables(self, variables):
        self.means = variables["means"]
        self.Pi = variables["Pi"]

In [27]:
task.trusted_data

In [28]:
names = [
    "blocks.0.hook_attn_out.output",
    # "blocks.0.attn.hook_attn_scores.output",
    # "blocks.0.attn.hook_q.output",
    "ln_final.hook_normalized.output",
]
detector = GradientWeightedDetector(names=names, output_func_for_grads=effect_prob_func)
detector.train(task, trusted_data=None, untrusted_data=None, save_path=None, batch_size=64)

AssertionError: trusted_data and untrusted_data must be None when passing a task

In [15]:
px.imshow(detector.Pi["blocks.0.hook_attn_out.output"].cpu(), binary_string=True)

NameError: name 'detector' is not defined